# Testing Notebook

### Import libraries

In [9]:
import sys
import os

# Manually specify the path to the 'src' directory
src_path = os.path.abspath(os.path.join(os.getcwd(), '../src'))

# Add the specified path to the system path
sys.path.append(src_path)

# Third-party libraries
import networkx as nx

# Local application/library imports
import compatibility_graph as compGraph
import circuit_gen as circuitGen
import max_clique as maxClique

### Parameters Configuration

In [10]:
num_modules       = 4
module_max_qubits = 4
module_max_gates  = 6
buffer_distance   = 1

### Random Circuit Generation and Compatibility Graph Computation

In [11]:
# Generate random circuit
circuit = circuitGen.RandomCircuit(num_modules, module_max_qubits, module_max_gates)
circuit.gen_random_circuit()

# Generate compatibility graph
comp_graph = compGraph.CompatibilityGraph(modules=circuit.modules, buffer_distance=buffer_distance)
comp_graph.generate_compatibility_graph()
graph = comp_graph.graph

The backend was not provided when the compatibility graph was generated, so by default, FakeGuadalupeV2 will be used as the backend.

In [ ]:
# Plot module of the circuit
circuit.modules[0].draw('mpl')

In [13]:
# Plot compatibility graph
# nx.draw(graph)

### Max Clique Computation

In [ ]:
# Find maximum clique
max_clique, max_clique_weight = maxClique.find_max_clique(graph.to_undirected())

# Plot maximum clique
nx.draw_planar(graph.subgraph(max_clique), with_labels=True)

# Print results
print(f"Maximum clique: {max_clique}")
print(f"Weight of maximum clique: {max_clique_weight}")

In [ ]:
backend   = comp_graph.backend
couplings = comp_graph.coupling_map.get_edges()
qc_graph  = nx.from_edgelist(couplings)

# Retrieve the qubits associated with each chosen layout
chosen_layouts = []
for vertex in max_clique:
    chosen_layouts.append(comp_graph.modules[vertex[0]][0][vertex[1]][0])
print(f"Qubit associated with each layout: {chosen_layouts}")

In [ ]:
# Plot the quantum circuit graph with the chosen layout for each mapped module
colors    = ['blue', 'red', 'green', 'yellow', 'orange', 'purple', 'brown', 'pink']
color_map = []

for node in qc_graph:
    in_layout = False
    for layoutIndex, layout in enumerate(chosen_layouts):
        if node in layout:
            color_map.append(colors[layoutIndex + 1])
            in_layout = True
            
    if not in_layout:
        color_map.append(colors[0])
            
nx.draw(qc_graph, node_color=color_map, with_labels=True)